# Main executeable

Results saved to results/

## Imports & API connection test & init

In [12]:
import praw                             #Reddit API
import nltk                             #natural language
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import yfinance as yf                   #yahoo finance
from auth import *                      #authentification details - create an auth.py file in code/ with authentification details
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
import requests         
from bs4 import BeautifulSoup as bs     #web scraper

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT)

print(reddit.read_only)

True


## Get resources
Scrape updated list of tickers in S&P500 

In [11]:
import re

def get_spy():
    url = 'https://www.slickcharts.com/sp500'
    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs(request.text, "lxml")

    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df = pd.read_html(str(stats))[0]
    #FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.

    df['% Chg'] = df['% Chg'].str.strip('()-%')

    df['% Chg'] = pd.to_numeric(df['% Chg'])

    df['Chg'] = pd.to_numeric(df['Chg'])

    #df["Company"] = df["Company"].str.strip(" ")

    return df


business_suffixes = ["Corp", "Corporation", "Inc", "LLC", "Limited", "Ltd", "Inc.", "Class A", "Class B", "Class C", ".", ","]


pattern = re.compile(rf'\s*(?:{"|".join(business_suffixes)})(?:[.,]?)\s*$', re.IGNORECASE)

def clean_company_name(name):
    #removes common business suffixes
    cleaned_name = pattern.sub('', name).strip()
    return cleaned_name


df = get_spy()
sp_tickers = df["Symbol"]   #list of tickers in sp500 as strings

sp_names = df["Company"]    #list of all constituents

cln_sps = []
for name in sp_names:
    cln_sps.append(clean_company_name(name))

#cln_sps = {clean_company_name(name) for name in sp_names}

print(len(sp_tickers))

#Sample output to test
for i in range(0, 25):
    print(sp_tickers[i])
    print(cln_sps[i])


# Subreddits to parse:
subreddits = ["wallstreetbets", "investing", "trading", "stocks", "stockmarket"]


503
MSFT
Microsoft
AAPL
Apple
NVDA
Nvidia
AMZN
Amazon.com
META
Meta Platforms, Inc.
GOOGL
Alphabet Inc.
GOOG
Alphabet Inc.
BRK.B
Berkshire Hathaway
LLY
Eli Lilly & C
JPM
Jpmorgan Chase & C
AVGO
Broadcom
XOM
Exxon Mobil
UNH
Unitedhealth Group Incorporate
V
Visa
TSLA
Tesla,
MA
Mastercard Incorporate
PG
Procter & Gamble Compan
JNJ
Johnson & Johnso
HD
Home Depot,
MRK
Merck & Co.,
COST
Costco Wholesale
ABBV
Abbvie
CVX
Chevron
CRM
Salesforce,
BAC
Bank of America


/var/folders/1c/4fc0bzrd6sb0t89qb2xmbr740000gn/T/ipykernel_6781/1054192475.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(stats))[0]


## Functions

In [18]:
def postSentiment(urlT):
    try:
        post = reddit.submission(url=urlT)
        pbody = post.selftext
        #print(post.title)
    except:
        return 0
    

    sia = SIA()
    body_sentiments = sia.polarity_scores(pbody)
    title_sentiment = sia.polarity_scores(post.title)

    avg_sentiment = (body_sentiments['compound']+title_sentiment['compound'])/2
    print(avg_sentiment)
    '''
    for key in snt.keys():
        print(key, snt[key])
    
    for key in sentiments.keys():
        print(key, sentiments[key])
    '''
    return avg_sentiment

postSentiment('https://www.reddit.com/r/wallstreetbets/comments/1cbrwwz/goog_the_guy_who_killed_yahoo_search_is_now/')

    
    


def commentSentiment(urlT):
    #given post that mentions ticker, will calculate average sentiment of comments to that post

    comments = [] 
    bodyComment = []

    #get comments from sub
    try:
        post = reddit.submission(url=urlT)
        comments = post.comments            #returns iterable CommentForest object
    except:
        return 0
    
    #save each comment to array
    for comment in comments:
        try: 
            bodyComment.append(comment.body)   
        except:
            return 0
    
    sia = SIA()
    results = []
    for line in bodyComment:
        scores = sia.polarity_scores(line)
        scores['headline'] = line

        results.append(scores)
    
    df = pd.DataFrame.from_records(results)
    df.head()
    df['label'] = 0
    
    try:
        df.loc[df['compound'] > 0.1, 'label'] = 1
        df.loc[df['compound'] < -0.1, 'label'] = -1
    except:
        return 0
    
    averageScore = 0
    position = 0
    while position < len(df.label)-1:
        averageScore = averageScore + df.label[position]
        position += 1

    averageScore = averageScore/len(df.label) 
    
    return(averageScore)



0.07040000000000002


## Computations

### Version 3
Iterate through each ticker in SP500, go through each subreddit analyzing each post that mentions the ticker. Measure the sentiment of the text in the post itself (if it exists) and calculate an average of sentiment of the comments in response to this post.

This cell produces a set of .csv files for each ticker with the following columns of data: 

Date (post), Sentiment of post, comment sentiment average, score, upvote ratio, number of crossposts, domain, intraday return, next day return, next to 5th trading day return, next to 10th trading return, next to 20th trading day return.

In [ ]:
##VERSION 3

# NOTE: Running this takes a lot of time. 

for ticker in sp_tickers:
    submission_statistics = []
    d = {}

    for subname in subreddits:
        for submission in reddit.subreddit(subname).search(ticker, limit=150):
            #if submission.domain != "self.wallstreetbets":
                #continue
            d = {}
            d['date'] = submission.created_utc

            d['num_comments'] = submission.num_comments
            d['comment_sentiment_average'] = commentSentiment(ticker, submission.url)
            if d['comment_sentiment_average'] == 0.000000:
                continue
            d['score'] = submission.score
            d['upvote_ratio'] = submission.upvote_ratio
            
            d['domain'] = submission.domain
            d['num_crossposts'] = submission.num_crossposts
            d['author'] = submission.author
            dates = format_time_add_day(d['date'])
            d['intraday_return'] = get_intraday_return(tickers[ticker], dates)
            d['next_day return'] = get_nextday_ret()
            d['latest_comment_date'] = latestComment(ticker, submission.url)
            submission_statistics.append(d)
        
        dfSentimentStocks = pd.DataFrame(submission_statistics)

        _timestampcreated = dfSentimentStocks["date"].apply(get_date)
        dfSentimentStocks = dfSentimentStocks.assign(timestamp = _timestampcreated)

        _timestampcomment = dfSentimentStocks["latest_comment_date"].apply(get_date)
        dfSentimentStocks = dfSentimentStocks.assign(commentdate = _timestampcomment)

        dfSentimentStocks.sort_values("latest_comment_date", axis = 0, ascending = True,inplace = True, na_position ='last') 

        dfSentimentStocks

        dfSentimentStocks.author.value_counts()
        dfSentimentStocks.to_csv("../results/" + ticker + "_SA.csv", index=False) 